In [213]:
import pandas as pd
import numpy as np
import datetime
import pyodbc
isdw = pyodbc.connect('DSN=ISDW')
qdat = pyodbc.connect('DSN=QDAT')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [66]:
processes = pd.read_csv("~/Downloads/process.csv",header=None)

In [55]:
def SQL_numtools(process):
    daysback = 500
    SQLQuery = """
    SELECT DISTINCT proc_Eqp_Id as tool
    FROM DMIW.ProcessedFact pf
    JOIN DMIW_SYSTEMS.MeasuredParm mp ON mp.measuredParmKey = pf.measuredParmKey
    JOIN DMIW_SYSTEMS.ProcessedWafer pw ON pw.processedWaferKey = pf.processedWaferKey
    WHERE Process_date >= (current date - {} days)
      AND Tech_id = '5H 14HP' AND Proc_pd_id LIKE '{}%' 
      AND UCASE(dcItem_Name) NOT IN ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN')
    """.format(daysback,process)
    df = pd.read_sql(SQLQuery,isdw)
    return df

In [56]:
process = []
num_tools = []
for item in processes[0]:
    tools = SQL_numtools(item)
    process.append(item)
    num_tools.append(len(tools))

In [58]:
df = pd.DataFrame(list(zip(process, num_tools)), 
               columns =['process', 'num_tools']) 

In [65]:
df[df.num_tools > 1]

,process,num_tools
0,C4-LTHIPS3,2
1,C4-RIEASHMTIN,4
3,CK-WETPTEOSCLN,4
4,CMPW5TTP,2
7,FRNFCVDSTANN55BP,2
8,FRNIRADFHP,2
11,FRNMHKHMUPP,2
13,I1-LTHIPS3,2
14,I2-LTHIPS3,2
15,I3-LTHIPS3,2


In [64]:
for i in df[df.num_tools == 1]['process']:
    print(i)

CMPWGATE5NGP
CMPWGATETU5NGP
FRNIRADNITNGP
FRNMHKHIGMLD551P
FRNMHKMLDNIT52P
INSACONHMTBP
INSCPLDTP
INSFCVDPOC55BP
INSMOLNITRMP
INSNITRMP
INSOXIDEAPP
INSOXIDEAQP
INSOXIDENGP
INSSICONICLNEGP
INSSICONIRECPOP
INSUDOXBIP
LTHIPS5DTP
MTLALAPP
MTLALEGP
MTLMOLDEGAS5RMP
MTLRMGTiNEGP
RIEFINSCUTFCP
RIEHMO5CTP
RIEMHKHRDMSK5PCP
RIEMHKOXIDEAPP
RIEMHKOXIDEAQP
RIEMHKOXIDENGP
RIESIARCFQP
RTPDPNHDEPIpPHP
RTPGATEASI5ACP
RTPTINASI5APP
WETStripNiTBP


In [214]:
def SQL_pulltools(process):
    daysback = 500
    SQLQuery = """
    SELECT DISTINCT e.EQP_ID AS TOOL
    FROM DMIW.CLAIM_FACT cf
    JOIN DMIW_SYSTEMS.PROCESS_DEFINITION pd on cf.processdefkey = pd.processdefkey
    JOIN DMIW_SYSTEMS.EQUIPMENT e on cf.equipmentkey = e.equipmentkey
    JOIN DMIW_SYSTEMS.PRODUCT p on cf.productkey = p.productkey
    JOIN DMIW_SYSTEMS.MAINPD mpd on cf.mainpdkey = mpd.mainpdkey
    JOIN DMIW_SYSTEMS.CLAIM c on cf.claimkey = c.claimkey
    WHERE cf.CLAIM_TS_LOCAL >= (current date - {} days)
      AND pd.PD_ID LIKE '{}%'
      --AND (p.lot_id LIKE '8IQV08008%' OR p.lot_id LIKE '19365HUS001%')
      AND (p.lot_id LIKE '8IQV09005%' OR p.lot_id LIKE '19355HUS013%')
      AND e.EQUIPMENTKEY NOT IN ('-1')
      AND UCASE(e.EQP_TYPE) NOT IN ('AUTO')
    """.format(daysback,process)
    df = pd.read_sql(SQLQuery,isdw)
    return df

In [215]:
process = []
tools_used = []
for item in processes[0]:
    tools = SQL_pulltools(item)
    process.append(item)
    tools_used.append(tools.TOOL[0]) if list(tools.TOOL) else tools_used.append(None)

In [216]:
df = pd.DataFrame(list(zip(process, tools_used)), 
               columns =['process', 'tool_used']) 

In [218]:
df.to_csv("~/Downloads/possible_culprits1.csv",index=False)

In [217]:
df

,process,tool_used
0,C4-LTHIPS3,ALC2361
1,C4-RIEASHMTIN,ETX29402
2,CK-WETPTEOSCLN,SNK29100
3,CMPW5TTP,EKD2
4,FRNFCVDSTANN55BP,BB02
...,...,...
63,WETMHKTiNBSRM57P,DC14
64,WETNITPSTCLNUPP,DCD2
65,WETPSTSPCR2BPP,DC14
66,WETRinsRwkMHKNGP,None
